In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [2]:
all_features = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "enaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp",\
                "retweet_with_comment_timestamp", "like_timestamp"]
training = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8", names = all_features, sep="\x01", nrows = 1000000)

In [31]:
training

,text_ tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,enaged_with_user_id,...,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,retweet_with_comment_bool,present_links_bool
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...,NaN,D4D1EBDE74F74C5DA529959AF979625C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,...,600,False,1520948869,False,NaN,NaN,NaN,NaN,0.0,0
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,BFB529DAB6D384EB83E899A72AB3830D,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,...,77,False,1569692352,True,NaN,NaN,NaN,NaN,0.0,0
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...,NaN,519078C7834E9642508F72A6C2D0F3B7,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,...,29,False,1568107028,False,NaN,NaN,NaN,1.581247e+09,0.0,788ABAA22F1C4A9D186510C1E3E81F58
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,...,30,False,1568107028,False,NaN,NaN,NaN,NaN,0.0,C06B84A10A75AE4BF4A102AB45CC2ABF
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...,NaN,89C1298C55EB3D68E2784F0BFB69E6F8,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,...,30,False,1568107028,False,NaN,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,101\t56898\t137\t14325\t10371\t10738\t24813\t1...,NaN,AF66180E09613048A06693E1DA25F3D5,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581022874,29B8298441C2D4E5D1AEC50D7BEF3712,...,129,False,1528034964,True,NaN,NaN,NaN,NaN,0.0,0
999996,101\t56898\t137\t33787\t11403\t23315\t25905\t1...,NaN,4C83E289F6668FCF6B4ABF223769D7A8,Video,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581474785,AD68074E50E59657A5822ED32C87897A,...,190,False,1242418375,False,NaN,NaN,NaN,NaN,0.0,0
999997,101\t10117\t11545\t13646\t75877\t70615\t14120\...,NaN,E05A382AEFE3F44518C98233CF439784,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581545832,790D519ED5D869AB368026BBAD8BB5CF,...,37,False,1507734862,False,NaN,NaN,NaN,NaN,0.0,0
999998,101\t56898\t137\t41626\t168\t178\t10147\t131\t...,NaN,7F6D93DF58E7B05DD51A81A99A0D6C65,Photo\tPhoto,NaN,NaN,Retweet,125C57F4FA6D4E110983FB11B52EFD4E,1581260734,08AA7A7638F95B19CFC170588DE73F04,...,354,False,1348229207,False,NaN,1.581267e+09,NaN,NaN,0.0,0


In [4]:
training['retweet_with_comment_bool'] = training.retweet_with_comment_timestamp.fillna(0)
training.loc[training.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0

In [30]:
training['present_links_bool'] = training.present_links.fillna('0')
# training.loc[training.present_links != '0', 'present_links_bool'] = '1'

In [6]:
(training.retweet_with_comment_bool == 1.0).sum()

7750

In [27]:
training.columns

Index(['text_ tokens', 'hashtags', 'tweet_id', 'present_media',
       'present_links', 'present_domains', 'tweet_type', 'language',
       'tweet_timestamp', 'enaged_with_user_id',
       'engaged_with_user_follower_count', 'engaged_with_user_following_count',
       'engaged_with_user_is_verified', 'engaged_with_user_account_creation',
       'enaging_user_id', 'enaging_user_follower_count',
       'enaging_user_following_count', 'enaging_user_is_verified',
       'enaging_user_account_creation', 'engagee_follows_engager',
       'reply_timestamp', 'retweet_timestamp',
       'retweet_with_comment_timestamp', 'like_timestamp',
       'retweet_with_comment_bool', 'present_links_bool'],
      dtype='object')

In [32]:
training_subset = training[['present_media', 'present_links', 'present_domains', 'tweet_type', 'language',\
                     'enaged_with_user_id', 'engaged_with_user_is_verified', 'enaging_user_id',\
                    'enaging_user_is_verified', 'engagee_follows_engager', 'present_links_bool', 'retweet_with_comment_bool']]

In [33]:
training_subset

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool,retweet_with_comment_bool
0,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,False,0000006C3074607050F1339DDCB890BB,False,False,0,0.0
1,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,False,000005C520010F8917EEAB6F5B6EC1C4,False,True,0,0.0
2,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,788ABAA22F1C4A9D186510C1E3E81F58,0.0
3,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,False,000006829BEADA9EEA695CF0C334B426,False,False,C06B84A10A75AE4BF4A102AB45CC2ABF,0.0
4,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,29B8298441C2D4E5D1AEC50D7BEF3712,False,0DD2A774F7679AD1FD5912173D3760FC,False,True,0,0.0
999996,Video,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,AD68074E50E59657A5822ED32C87897A,True,0DD2A7E8AEF35BD0EB83FFA4504F2075,False,False,0,0.0
999997,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,790D519ED5D869AB368026BBAD8BB5CF,True,0DD2A82452C83CD3F825E73498EBD1DE,False,False,0,0.0
999998,Photo\tPhoto,NaN,NaN,Retweet,125C57F4FA6D4E110983FB11B52EFD4E,08AA7A7638F95B19CFC170588DE73F04,False,0DD2A83B6837B1FA23CEA85748175A46,False,False,0,0.0


In [34]:
X = training_subset.drop(['retweet_with_comment_bool'], axis = 1)
y = training_subset[['retweet_with_comment_bool']]

In [35]:
X.present_links_bool.nunique()

108173

In [22]:
X

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool
0,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,False,0000006C3074607050F1339DDCB890BB,False,False,0
1,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,False,000005C520010F8917EEAB6F5B6EC1C4,False,True,0
2,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,788ABAA22F1C4A9D186510C1E3E81F58
3,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,False,000006829BEADA9EEA695CF0C334B426,False,False,C06B84A10A75AE4BF4A102AB45CC2ABF
4,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,29B8298441C2D4E5D1AEC50D7BEF3712,False,0DD2A774F7679AD1FD5912173D3760FC,False,True,0
999996,Video,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,AD68074E50E59657A5822ED32C87897A,True,0DD2A7E8AEF35BD0EB83FFA4504F2075,False,False,0
999997,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,790D519ED5D869AB368026BBAD8BB5CF,True,0DD2A82452C83CD3F825E73498EBD1DE,False,False,0
999998,Photo\tPhoto,NaN,NaN,Retweet,125C57F4FA6D4E110983FB11B52EFD4E,08AA7A7638F95B19CFC170588DE73F04,False,0DD2A83B6837B1FA23CEA85748175A46,False,False,0


In [36]:
y

,retweet_with_comment_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
999995,0.0
999996,0.0
999997,0.0
999998,0.0


In [37]:
X = X.fillna('unknown')

In [38]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
fit = X.apply(lambda x: d[x.name].fit_transform(x))

In [39]:
fit

,present_media,present_links,present_domains,tweet_type,language,enaged_with_user_id,engaged_with_user_is_verified,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,present_links_bool
0,13,108172,23212,2,11,475230,0,0,0,0,0
1,3,108172,23212,2,46,68323,0,1,0,1,0
2,13,50710,12361,2,11,348102,0,2,0,0,50711
3,13,81120,17290,2,11,419651,0,2,0,0,81121
4,13,108172,23212,2,11,348102,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
999995,13,108172,23212,1,53,91469,0,656112,0,1,0
999996,9,108172,23212,1,53,381902,1,656113,0,0,0
999997,3,108172,23212,2,53,266472,1,656114,0,0,0
999998,4,108172,23212,1,4,18997,0,656115,0,0,0


In [40]:
y

,retweet_with_comment_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
999995,0.0
999996,0.0
999997,0.0
999998,0.0


In [41]:
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(fit, y)

SelectKBest(k='all', score_func=<function chi2 at 0x7fe3d1245950>)

In [42]:
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
# pyplot.bar([fit[i] for i in range(len(fs.scores_))], fs.scores_)
# pyplot.show()

Feature 0: 2.653269
Feature 1: 119859.479605
Feature 2: 6246.099652
Feature 3: 2.032464
Feature 4: 3267.581957
Feature 5: 101071.590757
Feature 6: 96.076845
Feature 7: 917.417222
Feature 8: 72.270157
Feature 9: 375.287724
Feature 10: 1676157.313163


In [43]:
fit.columns[10]

'present_links_bool'

In [22]:
fit.columns[7]

'enaging_user_id'